<a href="https://colab.research.google.com/github/singhnidh/machine-learning-lab/blob/main/Classify_newsgroup_Topics_with_Support_Vector_Machines_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import fetch_20newsgroups
groups = fetch_20newsgroups()

In [ ]:
data_train = fetch_20newsgroups(subset='train', random_state=21)
train_label = data_train.target
data_test = fetch_20newsgroups(subset='test', random_state=21)
test_label = data_test.target
len(data_train.data), len(data_test.data), len(test_label)

(11314, 7532, 7532)

In [ ]:
import numpy as np
np.unique(test_label)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [ ]:
import nltk
nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


True

In [ ]:
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import names

all_names = names.words()
WNL = WordNetLemmatizer()
def clean(data):
    cleaned = defaultdict(list)
    count = 0
    for group in data:
        for words in group.split():
            if words.isalpha() and words not in all_names:
                cleaned[count].append(WNL.lemmatize(words.lower()))
        cleaned[count] = ' '.join(cleaned[count])
        count +=1
    return(list(cleaned.values()))

In [ ]:
nltk.download('wordnet')
x_train = clean(data_train.data)
x_train[0]

[nltk_data] Downloading package wordnet to /root/nltk_data...


'bouncing lymenet lehigh university the following address are on the lymenet mailing but are rejecting since the list server originally accepted these address i assume these address have since been improperly functioning mail gateway might also be if you are listed here and would still like to remain on the please write to i will remove these address from the list before the next newsletter go a a general please remember to from all your mailing list before your account is this will save the listserv maintainer from many box lehigh university'

In [ ]:
len(x_train)

11314

In [ ]:
x_test = clean(data_test.data)
len(x_test)

7532

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(stop_words='english', max_features=1000)
X_train = tf.fit_transform(x_train)
X_test = tf.transform(x_test)
X_train.shape, X_test.shape

((11314, 1000), (7532, 1000))

#**SVC**

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc_lib = SVC(kernel = 'linear')
parameters = {'C' : (0.5,1.0,10,100)}

In [ ]:
from sklearn.model_selection import GridSearchCV
import timeit

grid_search1 =GridSearchCV(svc_lib, parameters, n_jobs = -1, cv = 3)
start_time = timeit.default_timer()
grid_search1.fit(X_train, train_label)
final = timeit.default_timer()-start_time
print("Execution Time : ",final)

Execution Time :  179.379478887


In [ ]:
print(grid_search1.best_params_)
print(grid_search1.best_score_)

{'C': 1.0}
0.7213187626843581


In [ ]:
grid_search_best1 = grid_search1.best_estimator_
accur1 = grid_search_best1.score(X_test, test_label)
print(accur1)

0.6259957514604355


#**Linear SVC**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
import timeit

linear_svc = LinearSVC()
parameters = {'C': (0.5, 1, 10,100)}

grid_search2 =GridSearchCV(linear_svc, parameters, n_jobs = -1, cv = 3)
start_time = timeit.default_timer()
grid_search2.fit(X_train, train_label)
final = timeit.default_timer()-start_time
print("Execution Time : ",final)

Execution Time :  9.532145298000046


In [ ]:
print(grid_search2.best_params_)
print(grid_search2.best_score_)

{'C': 0.5}
0.7240587621538075


In [ ]:
grid_search_best2 = grid_search2.best_estimator_
accur2 = grid_search_best2.score(X_test, test_label)
accur2

0.6303770578863516

#**Model Tuning -> Linear SVC**

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tf_id', TfidfVectorizer(stop_words = "english")), ('svm_im', LinearSVC())])
pipeline

parameter = {'tf_id__max_features' : (100,1000, 2000, 8000),
             'tf_id__max_df' : (0.25, 0.5),
             'tf_id__smooth_idf' : (True, False),
             'tf_id__sublinear_tf' : (True, False)
}

In [ ]:
grid_search = GridSearchCV(pipeline, parameter,cv = 3)
grid_search.fit(x_train, train_label)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tf_id',
                                        TfidfVectorizer(stop_words='english')),
                                       ('svm_im', LinearSVC())]),
             param_grid={'tf_id__max_df': (0.25, 0.5),
                         'tf_id__max_features': (100, 1000, 2000, 8000),
                         'tf_id__smooth_idf': (True, False),
                         'tf_id__sublinear_tf': (True, False)})

In [ ]:
print(grid_search.best_params_)

{'tf_id__max_df': 0.25, 'tf_id__max_features': 8000, 'tf_id__smooth_idf': False, 'tf_id__sublinear_tf': True}


In [ ]:
print(grid_search.best_score_)

0.8700729197980644
